<a href="https://colab.research.google.com/github/MStamirski/Faces/blob/main/Dataset_prep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from PIL import Image, ImageFilter

In [ ]:
def get_labels():
  return pd.read_csv("faces-labels.csv", sep=';')

In [ ]:
def get_faces(labels, folder):
  faces_output = []
  sex_output = []
  hair_output = []
  glasses_output = []
  hat_output = []
  beard_output = []

  # indexes of faces to alter
  to_flip = (628, 746)
  to_rotater = (747, 867)
  to_rotatel = (868, 1023)
  to_narrow = (1024, 1205)
  to_widen = (1206, 1349)

  for face_nr in labels['face_nr']:

    print(f"\rProcessing face nr {face_nr}", end="")

    file = 'face'+str(face_nr)+'.png'
    face = Image.open(folder+file)
    face = face.convert("L")

    # altering faces with specific indexes
    if to_flip[0] <= face_nr <= to_flip[1]:
      face = face.transpose(Image.FLIP_LEFT_RIGHT)
    elif to_rotater[0] <= face_nr <= to_rotater[1]:
      face = face.rotate(338)
    elif to_rotatel[0] <= face_nr <= to_rotatel[1]:
      face = face.rotate(22)
    elif to_narrow[0] <= face_nr <= to_narrow[1]:
      face = face.resize((50, 64))
      background_arr = np.array([[0.0 for x in range(64)] for x in range(64)])
      background = Image.fromarray(background_arr)
      background.paste(face, ((7, 0)))
      face = background.convert("L")
    elif to_widen[0] <= face_nr <= to_widen[1]:
      face = face.resize((78, 64))
      face = face.crop((7, 0, 71, 64))

    # augmenting faces set
    face_enlarged = face.resize((84, 84))
    face_enlarged = face_enlarged.crop((10, 10, 74, 74))
    face_blured = face.filter(ImageFilter.BoxBlur(1))
    face_enlarged_blured = face_enlarged.filter(ImageFilter.BoxBlur(1))

    # creating arrays of faces and labels
    labels_list = labels[labels['face_nr']==face_nr].to_numpy()[0, 1:].tolist()

    for face_to_add in [face, face_enlarged, face_blured, face_enlarged_blured]:
      face_arr = np.asarray(face_to_add)
      faces_output.append(face_arr)

      sex_output.append(labels_list[0])
      hair_output.append(labels_list[1])
      glasses_output.append(labels_list[2])
      hat_output.append(labels_list[3])
      beard_output.append(labels_list[4])

  print(f"\nCreated dataset contains {len(sex_output)} faces.")

  return np.array(faces_output), np.array(sex_output), np.array(hair_output), np.array(glasses_output), np.array(hat_output), np.array(beard_output)